In [1]:
import pandas_ta as ta
import pandas as pd
import numpy as np
from datetime import datetime
import main_functions
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
import warnings
# from lightweight_charts import Chart
# from lightweight_charts.widgets import JupyterChart
# import mplfinance as mpf
# import yfinance as yf
# Filter out the FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

In [18]:
import importlib
# If you want to update the module:
importlib.reload(main_functions)
from main_functions import *

In [98]:
# exchange = krakenActive(mode)

In [3]:
# Initialize notebook mode
init_notebook_mode(connected=True)

In [4]:
pd.set_option('display.max_rows', 500)  # Set the maximum number of rows to display
pd.set_option('display.max_columns', 40) 

# Defining the variables

In [5]:
ticker = "BTCUSDT"
timeframe = "1d"
start = "2019-09-08"
start_date = datetime.strptime(start, "%Y-%m-%d")
end = str(datetime.now())
# Calculate the difference in days
day = (datetime.now() - start_date).days
direction = 'Both' # other options for direction 'Long' or 'Short'

In [6]:
max_sw_cnt_l = 3
max_sw_cnt_s = 3

In [7]:
df = getdata(ticker, timeframe, day)

In [8]:
def calculate_gann_signals(df, max_sw_cnt = max_sw_cnt_l, exit_perc = 80/100, side = "long"):
    calculate_candle_type(df)
    
    # Initialize p_cnt with a list containing the initial value (0) for the first row
    p_cnt_values = [0]

    # Iterate over the rows of the DataFrame starting from the second row (index 1)
    for i in range(1, len(df)):
        # Get the current candlestick type for the current row
        current_candle_type = df['candle_type'].iloc[i]

        # Get the previous sw_cnt value (p_cnt) for the current row
        previous_sw_cnt = p_cnt_values[i - 1]

        # Initialize a variable to store the new sw_cnt value
        new_sw_cnt = None

        # Check various conditions to determine the new sw_cnt value
        if (current_candle_type == "up_bar" and previous_sw_cnt < 0) or (current_candle_type == "outside_dn_up" and previous_sw_cnt < 0):
            new_sw_cnt = 1
        elif current_candle_type == "up_bar" and previous_sw_cnt < max_sw_cnt:
            new_sw_cnt = previous_sw_cnt + 1
        elif current_candle_type == "up_bar" and previous_sw_cnt == max_sw_cnt:
            new_sw_cnt = max_sw_cnt
        elif (current_candle_type == "down_bar" and previous_sw_cnt > 0) or (current_candle_type == "outside_up_dn" and previous_sw_cnt > 0):
            new_sw_cnt = -1
        elif current_candle_type == "down_bar" and previous_sw_cnt > -max_sw_cnt:
            new_sw_cnt = previous_sw_cnt - 1
        elif current_candle_type == "down_bar" and previous_sw_cnt == -max_sw_cnt:
            new_sw_cnt = -max_sw_cnt
        elif current_candle_type == "outside_dn_up" and previous_sw_cnt > 0:
            new_sw_cnt = previous_sw_cnt
        elif current_candle_type == "outside_up_dn" and previous_sw_cnt < 0:
            new_sw_cnt = previous_sw_cnt
        else:
            # If none of the conditions are met, keep the sw_cnt unchanged
            new_sw_cnt = previous_sw_cnt

        # Append the new sw_cnt value to the list of sw_cnt values
        p_cnt_values.append(new_sw_cnt)

    # Check if the length of sw_cnt values matches the number of rows in df
    if len(p_cnt_values) == len(df):
        # Assign the list of sw_cnt values to a new column 'sw_cnt' in df
        df['sw_cnt'] = p_cnt_values
    else:
        print("Length mismatch error between p_cnt_values and df.")

    # Initialize sw_trend with NaN
    df['sw_trend'] = np.nan

    # Create mask1 and mask2
    mask1 = (
        ((df['sw_cnt'] == -(max_sw_cnt)) |
         ((df['sw_cnt'] == 1) & (df['candle_type'] == 'outside_dn_up'))) &
        (df['sw_cnt'].shift(1) == -(max_sw_cnt-1))  # & (df['sw_trend'].shift(1) != df['sw_trend'])
    )

    mask2 = (
        ((df['sw_cnt'] == (max_sw_cnt)) |
         ((df['sw_cnt'] == -1) & (df['candle_type'] == 'outside_up_dn'))) &
        (df['sw_cnt'].shift(1) == (max_sw_cnt-1))  # & (df['sw_trend'].shift(1) != df['sw_trend'])
    )

    # Update sw_trend based on mask1 and mask2
    df.loc[mask1, 'sw_trend'] = -1
    df.loc[mask2, 'sw_trend'] = 1

    # Forward fill the sw_trend column to carry forward the last value
    df['sw_trend'].ffill(inplace=True)

    trend_cnt = [0]

    # Loop through the DataFrame index
    for i in range(1, len(df)):
        if df['sw_trend'].iloc[i - 1] == df['sw_trend'].iloc[i]:
            trend_cnt.append(trend_cnt[i - 1] + 1)
        else:
            trend_cnt.append(1)
    # Append a 0 at the beginning to match the length of the DataFrame
    trend_cnt.insert(0, 0)

    df['trend_cnt'] = trend_cnt[:-1]

    # Initialize sw_top column with False
    df['sw_top'] = False
    df['sw_bottom'] = False

    # Calculate the maximum High value of past n candles when mask1 is true
    for i in range(len(df)):
        if mask1[i] & (df['sw_trend'].iloc[i - 1] != df['sw_trend'].iloc[i]):
            # Calculate the maximum High and its index
            high_range = df.loc[
                df.index[max(0, i - int(df['trend_cnt'][i]))]:df.index[i]]['High']
            max_high = high_range.max()
            max_high_index = high_range.idxmax()

            # Mark the row where max_high occurs as True
            df.at[max_high_index, 'sw_top'] = True

        elif mask2[i] & (df['sw_trend'].iloc[i - 1] != df['sw_trend'].iloc[i]):
            # Calculate the maximum High and its index
            low_range = df.loc[
                df.index[max(0, i - int(df['trend_cnt'][i]))]:df.index[i]]['Low']
            min_low = low_range.min()
            min_low_index = low_range.idxmin()

            # Mark the row where max_high occurs as True
            df.at[min_low_index, 'sw_bottom'] = True

    df['sw_high_price'] = np.where(
        df['sw_top'] == True, df['High'], np.nan)
    df['sw_high_price'].fillna(method='ffill', inplace=True)
    df['sw_low_price'] = np.where(
        df['sw_bottom'] == True, df['Low'], np.nan)
    df['sw_low_price'].fillna(method='ffill', inplace=True)

    # Filter rows where sw_top is True and calculate sw_highs
    df_tops = df[df['sw_top'] == True].copy()
    df_tops['sw_highs'] = np.where(
        df_tops['High'] > df_tops['High'].shift(1), "HH", "LH")

    # Filter rows where sw_bottom is True and calculate sw_lows
    df_bottoms = df[df['sw_bottom'] == True].copy()
    df_bottoms['sw_lows'] = np.where(
        df_bottoms['Low'] < df_bottoms['Low'].shift(1), "LL", "HL")

    # Concatenate the DataFrames and select the desired columns
    df_swings = pd.concat([df_tops, df_bottoms], axis=1)

    df_swings['trend'] = np.nan
    df_swings['trend'] = np.where(((df_swings['sw_highs'] == 'HH') & (df_swings['sw_lows'].shift(1) == 'HL')) |
                                  ((df_swings['sw_lows'] == 'HL') & (df_swings['sw_highs'].shift(1) == 'HH')),
                                   "UP",
                                   np.where(((df_swings['sw_lows'] == 'LL') & (df_swings['sw_highs'].shift(1) == 'LH')) |
                                            ((df_swings['sw_highs'] == 'LH') & (df_swings['sw_lows'].shift(1) == 'LL')),
                                            "DOWN",
                                            np.where(((df_swings['sw_lows'] == 'LL') & (df_swings['sw_highs'].shift(1) == 'HL')) |
                                                     ((df_swings['sw_lows'] == 'LL') & (df_swings['sw_highs'].shift(1) == 'HH')) |
                                                     ((df_swings['sw_lows'] == 'HL') & (df_swings['sw_highs'].shift(1) == 'LH')) |
                                                     ((df_swings['sw_highs'] == 'LH') & (df_swings['sw_lows'].shift(1) == 'HL')) |
                                                     ((df_swings['sw_highs'] == 'HH') & (df_swings['sw_lows'].shift(1) == 'LL')),
                                                     "UNCERTAIN",
                                                     np.nan
                                                    )
                                                 )
                                             )


    df_swings = df_swings[["sw_highs", "sw_lows", "trend"]]

    df_ffill = pd.concat([df, df_swings], axis=1)

    # Define the columns to be copied
    columns_to_copy = ["sw_highs", "sw_lows", "trend"]

    # Forward fill columns in df_ffill
    df_ffill[columns_to_copy] = df_ffill[columns_to_copy].fillna(method='ffill')

    # Drop the columns to be replaced from df
    for column in columns_to_copy:
        if column in df.columns:
            df.drop(columns=column, inplace=True)

    # Update the original DataFrame with the filled columns
    df = pd.concat([df, df_ffill[columns_to_copy]], axis=1)

    df['trend'] = np.where(
                            ((df['sw_lows'] == 'HL') & (df['High'] > df['sw_high_price'])),
                               "UP",
                               np.where(((df['sw_highs'] == 'LH') & (df['Low'] < df['sw_low_price'])),
                                        "DOWN",
                                        df['trend']
                                       )
                            )
    # df['trend'].fillna(method='ffill', inplace=True)
    if side == "long":
        df[f'tsl_{side}'] = df['sw_low_price'].shift(1)
        df[f'{side}_entry'] = df['sw_high_price'].shift(1)
        
        df[f"{side}_Signal"] = np.where(
                            ((df['sw_lows'] == "HL") | ((df['sw_highs'] == "HH") & (df['sw_lows'] == "HL"))) & 
                            ((df['High'] > df['sw_high_price'].shift(1)) &
                            (df['High'].shift(1) < df['sw_high_price'].shift(1)) &
                            ((df['trend'].shift(1) == "UNCERTAIN") | (df['trend'] == "UP"))),
                            True,
                            False
                            )
        
        df[f"{side}_Exit"] = np.where((df['sw_highs'] == "LH") & 
                            (df['sw_trend'].shift(1) == 1.0) &
                            (df['sw_trend'] == -1.0) &
                            (df['trend'] == "UNCERTAIN"),  
                            ((df['sw_high_price'] - df['sw_low_price'])*exit_perc + df['sw_low_price']), 
                                np.nan)
        
        df["pi_top"] = np.where(
                            (df['Open'].rolling(window=111).mean()) > (df['Open'].rolling(window=350).mean() * 2), 
                            True, 
                            False
                            )
    
        
    else:
        df[f'tsl_{side}'] = df['sw_high_price'].shift(1)
        df[f'{side}_entry'] = df['sw_low_price'].shift(1)
        
        df[f"{side}_Signal"] = np.where(
                            ((df['sw_highs'] == "LH") | ((df['sw_lows'] == "LL") & (df['sw_highs'] == "LH"))) & 
                            (df['Low'] < df['sw_low_price'].shift(1)) &
                            (df['Low'].shift(1) > df['sw_low_price'].shift(1)) &
                            ((df['trend'].shift(1) == "UNCERTAIN") | (df['trend'] == "DOWN")),  
                            True, 
                            False
                            )

        df[f"{side}_Exit"] = np.where((df['sw_lows'] == "HL") & 
                            (df['sw_trend'].shift(1) == -1.0) &
                            (df['sw_trend'] == 1.0) &
                            (df['trend'] == "UNCERTAIN"),  
                            (df['sw_high_price'] - (df['sw_high_price'] - df['sw_low_price'])*exit_perc), 
                                np.nan)    

        df["pi_bottom"] = np.where(
                            (df['Close'].rolling(window=550).mean() > df['Close'].rolling(window=250).mean()), 
                            True, 
                            False
                            )
    
    st.write(df[300:500])
    
    return df

In [9]:
def backtest(df, ticker, direction="Both", commission=0.04/100, tp_perc=0, pi_exit = True, tsl_offset = 0.1/100):
    in_position = False
    buy_pos = False
    sell_pos = False

    results_df = pd.DataFrame()
    buydates, buyprices = [], []
    selldates, sellprices = [], []
    exit_types = []

    for index, row in df.iterrows():
    # ---------------------------------------------long position close check------------------------------
        if in_position and buy_pos:
            
            sl = row.tsl_long*(1-tsl_offset)
            if (row.Low <= sl):
                selldates.append(index)
                sellprices.append(row.Low)
                in_position = False
                buy_pos = False
                exit_types.append("SL Hit")
            elif (row.High >= tp) and (tp_perc != 0):
                selldates.append(index)
                sellprices.append(tp)
                in_position = False
                buy_pos = False
                exit_types.append("TP Hit")
            elif (row.pi_top) and (pi_exit):
                selldates.append(index)
                sellprices.append(row.High)
                in_position = False
                buy_pos = False
                exit_types.append("Pi Cycle")
            elif row.long_Exit > 0:
                limit = row.long_Exit
            elif row.Low <= limit:
                selldates.append(index)
                sellprices.append(limit)
                in_position = False
                buy_pos = False
                exit_types.append("Limit price Hit due to uncertain trend")

    # ---------------------------------------------short position close check------------------------------
        elif in_position and sell_pos:
            sl = row.tsl_short*(1+tsl_offset)
            if (row.High >= sl):
                buydates.append(index)
                buyprices.append(row.High)
                in_position = False
                sell_pos = False
                exit_types.append("SL Hit")
                
            elif (row.Low <= tp) and (tp_perc != 0):
                buydates.append(index)
                buyprices.append(tp)
                in_position = False
                buy_pos = False
                exit_types.append("TP Hit")
            elif (row.pi_bottom) and (pi_exit):
                buydates.append(index)
                buyprices.append(row.Low)
                in_position = False
                buy_pos = False
                exit_types.append("Pi Cycle")
            elif row.short_Exit > 0:
                limit = row.short_Exit
            elif row.High >= limit:
                buydates.append(index)
                buyprices.append(limit)
                in_position = False
                buy_pos = False
                exit_types.append("Limit price Hit due to uncertain trend")
                
    #         print(limit, in_position)
                
    # ======================================================================================================              
                
    # ---------------------------------------------long position entry check------------------------------
        if not in_position:
            if direction in ("Both", "Long") and row.long_Signal:
                buyprice = row.long_entry
                buydates.append(index)
                buyprices.append(buyprice)
                in_position = True
                buy_pos = True
                tp = buyprice * (1 + tp_perc)
                limit = np.nan
                tsl_offset = tsl_offset_pct/100 if tsl_offset_en == True else 0

            elif direction in ("Both", "Short") and row.short_Signal:
                sellprice = row.short_entry
                selldates.append(index)
                sellprices.append(sellprice)
                in_position = True
                sell_pos = True
                tp = sellprice / (1 + tp_perc)
                limit = np.nan
                tsl_offset = tsl_offset_pct/100 if tsl_offset_en == True else 0

                    

    if len(buydates) == 0:
        print(f"No trades were made for {ticker}.")
    else:
        profits = [(sell - buy) / buy - commission for sell, buy in zip(sellprices, buyprices)]
        returns = ((pd.Series(profits, dtype=float) + 1).prod() - 1) * 100
        wins = 0
        for i in profits:
            if i > 0:
                wins += 1
            i += 1
        winrate = round((wins / len(buydates)) * 100, 2)
        ct = min(len(buydates), len(selldates))

        # BTCUSDT buy and hold returns during the same period
        buy_hold_ret = (df['Close'][-1] - df['Open'][0]) / df['Open'][0] * 100

        results_df = pd.concat([results_df, pd.DataFrame({'ticker': f'{ticker}', 'returns': [returns], 'winrate': [winrate], 'trades': [ct], 'buy&hold_ret%': [buy_hold_ret]})])
        st.subheader('Backtest Results')
        st.write(f'{ticker}, winrate={winrate}%, returns={round(returns, 2)}%, no. of trades = {ct}, buy&hold_ret = {round(buy_hold_ret, 2)}%')

    # Return the trade data along with other results
    return {
        'buydates': buydates,
        'buyprices': buyprices,
        'selldates': selldates,
        'sellprices': sellprices,
        'profits': profits,
        'Exit Type' : exit_types
        # Other results...
    }, results_df

def displayTrades(direction="Both", **kwargs):
    st.write('Direction: ', direction)
    # Access the trade data and other results from kwargs
    buydates = kwargs['buydates']
    buyprices = kwargs['buyprices']
    selldates = kwargs['selldates']
    sellprices = kwargs['sellprices']
    profits = kwargs['profits']
    exit = kwargs['Exit Type']

    ct = min(len(buydates), len(selldates))
    
    # Create a DataFrame to store the trades
    dfr = pd.DataFrame()
    dfr['buydates'] = buydates[:ct]
    dfr['buyprice'] = buyprices[:ct]
    dfr['selldates'] = selldates[:ct]
    dfr['sellprice'] = sellprices[:ct]
    dfr['profits'] = (profits[:ct])
    dfr['commulative_returns'] = ((pd.Series(profits) + 1).cumprod())
    dfr['Exit Type'] = exit[:ct]
    
    # Add a column to indicate the trade side
    dfr['tradeSide'] = np.where(dfr['buydates'] < dfr['selldates'], 'Long', 'Short')
    
    return dfr

In [10]:
dfl = calculate_gann_signals(df, max_sw_cnt = max_sw_cnt_l, exit_perc = 80/100, side = "long")
dfs = calculate_gann_signals(df, max_sw_cnt = max_sw_cnt_s, exit_perc = 80/100, side = "short")
unique_columns = dfs.columns.difference(dfl.columns)
df = pd.concat([dfl, dfs[unique_columns]], axis=1)

2023-10-11 02:18:31.533 
  command:

    streamlit run c:\Users\abbAxi\kraken\k_env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [11]:
df

,Open,High,Low,Close,Volume,candle_type,sw_cnt,sw_trend,trend_cnt,sw_top,sw_bottom,sw_high_price,sw_low_price,sw_highs,sw_lows,trend,tsl_long,long_entry,long_Signal,long_Exit,pi_top,pi_bottom,short_Exit,short_Signal,short_entry,tsl_short
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-09-08,10000.00,10412.65,10000.00,10391.63,3096.291,,0,NaN,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,NaN,NaN
2019-09-09,10316.62,10475.54,10077.22,10307.00,14824.373,up_bar,1,NaN,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,NaN,NaN
2019-09-10,10307.00,10382.97,9940.87,10102.02,9068.955,down_bar,-1,NaN,1,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,NaN,NaN
2019-09-11,10094.27,10293.11,9884.31,10159.55,10897.922,down_bar,-2,NaN,1,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,NaN,NaN
2019-09-12,10163.06,10450.13,10042.12,10415.13,15609.634,up_bar,1,NaN,1,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-06,27398.30,28450.00,27156.00,27916.30,438445.904,outside_dn_up,2,1.0,5,False,False,27480.0,26050.0,LH,HL,UP,26050.0,27480.0,False,NaN,False,False,NaN,False,26050.0,27480.0
2023-10-07,27916.30,28018.70,27822.10,27943.00,117049.145,,2,1.0,6,False,False,27480.0,26050.0,LH,HL,UP,26050.0,27480.0,False,NaN,False,False,NaN,False,26050.0,27480.0
2023-10-08,27943.00,28082.00,27671.00,27901.20,178380.307,outside_up_dn,-1,1.0,7,False,False,27480.0,26050.0,LH,HL,UP,26050.0,27480.0,False,NaN,False,False,NaN,False,26050.0,27480.0


# Backtest

In [12]:
tp_exit = False  #----> streamlit input
tp_value = 38/100 #----> streamlit input
tp_perc = 0 if tp_exit == False else tp_value
direction = "Both"
pi_exit = True

tsl_offset_en = True
tsl_offset_pct = 0.1
tsl_offset = tsl_offset_pct/100 if tsl_offset_en == True else 0

# no use found in the tradingview script
# entry_offset_en = False
# entry_offset_pct = 0.1
# entry_offset = entry_offset_pct/100 if entry_offset_en == True else 0

In [13]:
results_data, result_df = backtest(df, ticker, direction="Both", commission=0.04/100, tp_perc=tp_perc, pi_exit = pi_exit)

In [14]:
result_df

,ticker,returns,winrate,trades,buy&hold_ret%
0,BTCUSDT,814.631673,56.52,22,174.153


In [15]:
dfr = displayTrades(direction="Both", **results_data)
dfr

,buydates,buyprice,selldates,sellprice,profits,commulative_returns,Exit Type,tradeSide
0,2019-10-25,8800.00,2019-10-23,7700.67,-0.125324,0.874676,SL Hit,Short
1,2020-01-06,7691.00,2020-03-08,7997.70,0.039478,0.909206,SL Hit,Long
2,2020-04-02,7290.00,2020-03-08,8409.00,0.153098,1.048404,SL Hit,Short
3,2020-04-02,6960.71,2020-07-13,9339.28,0.341314,1.406239,Limit price Hit due to uncertain trend,Long
4,2020-07-22,9470.00,2020-09-03,9901.16,0.045129,1.469701,SL Hit,Long
5,2020-10-10,11494.00,2020-09-03,11130.00,-0.032069,1.422570,SL Hit,Short
6,2020-10-10,11177.00,2021-04-13,63850.00,4.712224,8.126036,Pi Cycle,Long
7,2021-04-13,61950.00,2021-04-14,64986.11,0.048609,8.521035,Pi Cycle,Long
8,2021-06-14,41000.00,2021-05-13,46930.43,0.144245,9.750148,SL Hit,Short
9,2021-06-14,39470.00,2021-06-22,28780.01,-0.271238,7.105534,SL Hit,Long


## CANDLES CHART

In [19]:
plot_advanced_gann_swing_chart(df, dfr, visible_data_points=1500)

In [35]:
styled_df = df.copy()
styled_df.reset_index(inplace = True)
# Define custom CSS to make the header row sticky
css = [
    {
        'selector': 'thead tr',
        'props': 'position: sticky; top: 0; background-color: white; color: black;'
    }
]

# Apply the custom CSS to the DataFrame
styled_df = styled_df.style.set_table_styles(css)

# Display the styled DataFrame
styled_df

,timestamp,Open,High,Low,Close,Volume,candle_type,sw_cnt,sw_trend,trend_cnt,sw_top,sw_bottom,sw_high_price,sw_low_price,sw_highs,sw_lows,trend,tsl_long,tsl_short,LONG_Signal,SHORT_Signal,Long_Exit,Short_Exit,pi_top,pi_bottom
0,2019-09-08 00:00:00,10000.000000,10412.650000,10000.000000,10391.630000,3096.291000,,0,nan,0,False,False,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,False,False
1,2019-09-09 00:00:00,10316.620000,10475.540000,10077.220000,10307.000000,14824.373000,up_bar,1,nan,0,False,False,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,False,False
2,2019-09-10 00:00:00,10307.000000,10382.970000,9940.870000,10102.020000,9068.955000,down_bar,-1,nan,1,False,False,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,False,False
3,2019-09-11 00:00:00,10094.270000,10293.110000,9884.310000,10159.550000,10897.922000,down_bar,-2,nan,1,False,False,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,False,False
4,2019-09-12 00:00:00,10163.060000,10450.130000,10042.120000,10415.130000,15609.634000,up_bar,1,nan,1,False,False,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,False,False
5,2019-09-13 00:00:00,10414.960000,10440.550000,10153.510000,10341.340000,19252.721000,,1,nan,1,False,False,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,False,False
6,2019-09-14 00:00:00,10343.010000,10419.970000,10222.770000,10332.250000,20656.624000,,1,nan,1,False,False,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,False,False
7,2019-09-15 00:00:00,10333.470000,10359.200000,10024.810000,10302.220000,20536.255000,down_bar,-1,nan,1,False,False,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,False,False
8,2019-09-16 00:00:00,10302.000000,10353.810000,10080.700000,10249.270000,20185.637000,,-1,nan,1,False,False,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,False,False
9,2019-09-17 00:00:00,10257.300000,10270.630000,10136.770000,10186.520000,22519.607000,,-1,nan,1,False,False,nan,nan,nan,nan,nan,nan,nan,False,False,nan,nan,False,False


# Set Final Bot Parameters

In [36]:
# please put this bot parameters here

ticker = "BTCUSDT"
timeframe = "15m"
usdt_amount = 15

takeLong = True 
takeShort = True

mult = 1 # TP R:R MULTIPLYER                           ----> TO BE OPTIMIZED
risk = 15.0 # IF STOPLOSS > ?%  DONT ENTER!            ----> TO BE OPTIMIZED
HL = 20 # rolling high lows                            ----> TO BE OPTIMIZED

In [21]:
# Trades Display

# def displayTrades():
#     ct = min(len(buydates),len(selldates))
#     dfr =pd.DataFrame()
#     dfr['buydates']= buydates[:ct]
#     dfr['buyprice']= buyprices[:ct]
#     dfr['selldates'] = selldates[:ct]
#     dfr['sellprice'] = sellprices[:ct]
#     dfr['profits'] = (profits[:ct])
#     dfr['commulative_returns'] = ((pd.Series(profits) + 1).cumprod())
#     dfr['tradeSide'] = np.where(dfr['buydates'] < dfr['selldates'], 'Long', 'Short')
#     return dfr

